In [1]:
import pandas as pd
import json
import requests
from PIL import Image
import numpy as np
import os
from io import BytesIO

output_dir = ('../images5')

In [2]:
import pickle
infile = open(r'C:\Users\Leonardo\forestfire\pipeline\test20191215.pickle','rb')
df1 = pickle.load(infile)
infile.close()

In [5]:
import pickle
infile = open(r'C:\Users\Leonardo\forestfire\pipeline\validation20191215.pickle','rb')
df2 = pickle.load(infile)
infile.close()

In [6]:
import pickle
infile = open(r'C:\Users\Leonardo\forestfire\pipeline\train20191215.pickle','rb')
df3 = pickle.load(infile)
infile.close()

In [9]:
df_labeled = pd.concat([df1, df2, df3], axis = 0)

In [11]:
df_shuffled = df_labeled.sample(frac=1, random_state = 42)

In [12]:
train_ind = int(round(df_shuffled.shape[0]*0.6))
val_ind = int(round(df_shuffled.shape[0]*0.8))

In [13]:
df_train = df_shuffled[0:train_ind]
df_val = df_shuffled[train_ind : val_ind]
df_test = df_shuffled[val_ind :]

In [14]:
df_train.shape, df_val.shape, df_test.shape

((5132, 6), (1710, 6), (1711, 6))

In [15]:
date = '20191215'

In [16]:
with open(r'C:\Users\Leonardo\forestfire\pipeline\train' + date + '.pickle', 'wb') as handle:
    pickle.dump(df_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
with open(r'C:\Users\Leonardo\forestfire\pipeline\validation' + date + '.pickle', 'wb') as handle:
    pickle.dump(df_val, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
with open(r'C:\Users\Leonardo\forestfire\pipeline\test' + date + '.pickle', 'wb') as handle:
    pickle.dump(df_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
os.getcwd()

'C:\\Users\\Leonardo\\forestfire\\pipeline'

In [32]:
os.chdir(r'C:\Users\Leonardo\forestfire\Importing_from_Labelbox')

In [37]:
os.mkdir(output_dir)

In [38]:
df_train.head()

,ID,External ID,url,firesmokes_url,othersmokes_url,label
5446,ck26qatq5qxtz08383lt9k9km,2018-03-18 13-08-56_NAO.jpeg,https://storage.labelbox.com/cjw0uwr2ui0vm0828...,https://us-central1-labelbox-193903.cloudfunct...,https://us-central1-labelbox-193903.cloudfunct...,1.0
2888,ck1wow2alemv40794ubazlrl3,2018-03-14 06-15-45_NAO.jpeg,https://storage.labelbox.com/cjw0uwr2ui0vm0828...,https://us-central1-labelbox-193903.cloudfunct...,https://us-central1-labelbox-193903.cloudfunct...,1.0
7072,ck2jb9u393s9p07214zlg3ors,2018-03-13 19-48-01_NAO.jpeg,https://storage.labelbox.com/cjw0uwr2ui0vm0828...,https://us-central1-labelbox-193903.cloudfunct...,https://us-central1-labelbox-193903.cloudfunct...,0.0
469,ck1p46cohr3ld0725del4re3l,2018-03-15 15-08-11_SIM.jpeg,https://storage.labelbox.com/cjw0uwr2ui0vm0828...,https://us-central1-labelbox-193903.cloudfunct...,https://us-central1-labelbox-193903.cloudfunct...,1.0
7221,ck2mb47kl0uzs0721zciluur8,2018-03-02 19-27-52_NAO.jpeg,https://storage.labelbox.com/cjw0uwr2ui0vm0828...,https://us-central1-labelbox-193903.cloudfunct...,https://us-central1-labelbox-193903.cloudfunct...,0.0


In [75]:
def load_labelbox(df, output_dir):
# This section retrieves images and masks from Labelbox using the DataFrames defined

    try:
        for i in range(len(df)):
            print("Image: {}, ID: {}".format(df.iloc[i,1], df.iloc[i, 0]))
            response_og = requests.get(df.iloc[i, 2])
            img_og = Image.open(BytesIO(response_og.content))
            img_og.save(os.path.join(output_dir, '{}_original.png'.format(i)), format='png')

            img_shape = np.array(img_og).shape

            img_mask = np.zeros((img_shape[0], img_shape[1]))
            img1, img2 = np.zeros(img_mask.shape), np.zeros(img_mask.shape)

            threshold = 200

            response1 = requests.get(df.iloc[i, 3])
            response2 = requests.get(df.iloc[i, 4])
            img1 = Image.open(BytesIO(response1.content))
            img2 = Image.open(BytesIO(response2.content))

            img3 = np.where(np.array(img1)[:, :, 3] > threshold, 255, 0)
            img4 = np.where(np.array(img2)[:, :, 3] > threshold, 255, 0)

            img_final = img3 + img4 - np.where(img3 + img4 > 255, 255, 0)
            img = Image.fromarray(img_final.astype(np.uint8)).convert(mode='L', dither=Image.NONE)
            img.save(os.path.join(output_dir, '{}_mask.png'.format(i)), format='png')

    except (TypeError, TimeoutError, OSError):
        pass

In [51]:
os.chdir('..')

In [55]:
base_dir = os.path.join(os.getcwd(), 'images5')

In [57]:
base_dir

'C:\\Users\\Leonardo\\forestfire\\images5'

In [58]:
os.mkdir(os.path.join(base_dir, 'train'))
os.mkdir(os.path.join(base_dir, 'val'))
os.mkdir(os.path.join(base_dir, 'test'))

In [71]:
train_output_dir = r'C:\Users\Leonardo\forestfire\images5\train'
val_output_dir = r'C:\Users\Leonardo\forestfire\images5\val'
test_output_dir = r'C:\Users\Leonardo\forestfire\images5\test'

In [72]:
load_labelbox(df_val, val_output_dir)

Image: 2018-03-12 09-21-39_NAO.jpeg, ID: ck1v2alcezdnt07573n0lu3qv
Image: 30_R_19082018_142912_mp4_1090, ID: ck2j4xj380tan0944sy8rplod
Image: 2019-03-24 08-06-59_SIM.jpeg, ID: ck22a94afyv9107485dsunhyd
Image: 2018-03-15 22-46-58_SIM.jpeg, ID: ck1w3y2kpaluc0944ufaecn2b
Image: 2018-03-02 10-41-14_NAO.jpeg, ID: ck202te3m6pu90757qxk0ivdg
Image: 2018-03-20 11-11-52_NAO.jpeg, ID: ck2mzozpm5co60838heyylmme
Image: 2018-03-02 15-59-01_NAO.jpeg, ID: ck2ijeptv16gs0721rn9ppah5
Image: 2018-03-12 09-15-06_NAO.jpeg, ID: ck1tluz7caycx0721xldaof4o
Image: 2018-03-11 14-06-34_NAO.jpeg, ID: ck1vbcwq3v01e0838icvnvlfh
Image: 2018-03-19 16-51-57_NAO.jpeg, ID: ck28b9vouk58m08116974xfa4
Image: 2018-03-17 14-18-35_NAO.jpeg, ID: ck1wbqfyxbu8q0838v4y9juhu
Image: Outras_sem_incendio_mp4_70, ID: ck2i5yzxoy49s0944ibo50ebw
Image: 2018-03-20 09-56-29_NAO.jpeg, ID: ck23nj4r5qtba07576ezomkrt
Image: 2018-03-18 08-05-06_NAO.jpeg, ID: ck27vsujdx73n07944z3481kq
Image: Incendio_IP_30_R_19082018_142912_mp4_1512, ID: ck2ipi365

Image: 2018-03-14 06-34-59_NAO.jpeg, ID: ck1yuwta1hk7y07257df58j0y
Image: 2018-02-27 16-28-21_SIM.jpeg, ID: ck1wmp87ndvoh0794qa0z02cr
Image: 2018-11-02 09-35-05_NAO.jpeg, ID: ck29btq36trea0725nm0biq7m
Image: 2019-07-04 13-14-10_NAO.jpeg, ID: ck29733p7ahq70757104jhot1
Image: 2018-01-25 15-49-00_NAO.jpeg, ID: ck1qxq8xpjly807251e41q8do
Image: 2018-03-12 08-13-54_NAO.jpeg, ID: ck1p1voo04gro0944sk14vtz3
Image: 2018-03-02 16-31-01_NAO.jpeg, ID: ck1zsku0rrkmi0748cf8sqyps
Image: 2018-03-18 09-49-18_NAO.jpeg, ID: ck29cejcn2h4w0794kef1039e
Image: 2018-03-18 10-10-01_NAO.jpeg, ID: ck23owsj6pzrf0721mdt0kxbg
Image: 2019-02-14 19-41-04_SIM.jpeg, ID: ck23ceuwjler9072188w8ccys
Image: Incendio_IP_IP_30_R_19082018_142912_mp4_1299, ID: ck2iqli5lurkc0794094f3av7
Image: 2018-10-30 08-51-22_SIM.jpeg, ID: ck25ndi7d73d80748fzitsuhd
Image: 2018-03-16 08-57-28_NAO.jpeg, ID: ck1us1vglehsf0725j1tbymq9
Image: Incendio_IP_IP_30_R_19082018_142912_mp4_2205, ID: ck2jmrzr2k0vs08118akj1mqv
Image: 2018-03-06 08-41-52_NAO

Image: 2018-03-07 20-49-34_SIM.jpeg, ID: ck2205ji01epa0944usopzhc0
Image: Incendio_IP_IP_30_R_19082018_142912_mp4_2022, ID: ck2jbn0ub638u0757wmjoipxl
Image: 2018-03-04 07-59-26_SIM.jpeg, ID: ck1w892uhfvcz0721wxhbkyqv
Image: 2018-03-15 16-37-07_NAO.jpeg, ID: ck1rgl9rnk6wv0811er90cdzv
Image: 2018-03-17 10-45-36_NAO.jpeg, ID: ck1wihd8ne85v08381p9wk6xz
Image: 2018-03-20 06-24-38_NAO.jpeg, ID: ck2385t8aaprt0748kf7iurxy
Image: 2018-03-10 10-38-27_NAO.jpeg, ID: ck2i1dd46ehkg0811lbcw6da9
Image: 2019-05-27 08-42-21_SIM.jpeg, ID: ck2n3s2f67kxr08386kisd8ew
Image: 2018-03-14 10-11-50_NAO.jpeg, ID: ck1rwatml9l3b079459op8em0
Image: 2018-03-16 10-42-16_NAO.jpeg, ID: ck1tgckqtn1a70811qee42h67
Image: 2018-03-12 10-19-47_NAO.jpeg, ID: ck1rlp1nn1oty0794bpoe9lbk
Image: 2018-03-18 12-53-44_NAO.jpeg, ID: ck26ceh8bhyrj07480jv4ffd9
Image: 2018-03-02 05-28-27_NAO.jpeg, ID: ck1v9v6ghehmh08116letfui6
Image: 2018-03-14 13-20-07_NAO.jpeg, ID: ck1ugv10mlbs60944p4he9l3x
Image: 2018-03-02 19-25-37_NAO.jpeg, ID: ck2jc

Image: 2018-03-07 20-44-13_SIM.jpeg, ID: ck20pmwk9la180721jxm057tk
Image: Outras_sem_incendio_mp4_92, ID: ck2jmi86w55mh0721z8wcmvbj
Image: 2018-03-20 16-42-08_NAO.jpeg, ID: ck28necndzp490794wez8tg7l
Image: 2018-03-04 23-11-54_NAO.jpeg, ID: ck1wof8lnj4yq09441gqv564s
Image: 2018-03-20 09-28-37_NAO.jpeg, ID: ck29bwimlnqaa0811qq2k5iyw
Image: 2018-03-16 14-38-39_NAO.jpeg, ID: ck1wq1uw9go7l083844bswf1a
Image: 2018-03-14 10-56-17_NAO.jpeg, ID: ck1p3a95y4otv0944wgj7wsxw
Image: 2018-03-18 07-52-45_NAO.jpeg, ID: ck22kxuklnxrj0811h8stn16i
Image: Incendio_IP_10_R_17072018_103000_mp4_516, ID: ck2iph73rsy8u0748tsdwdbqn
Image: 2018-03-16 09-17-41_NAO.jpeg, ID: ck1xa9t72pfhc072140y61045
Image: 2018-01-10 10-59-44_SIM.jpeg, ID: ck1usutlxoju10838i4qycfe1
Image: 2018-03-13 18-14-39_SIM.jpeg, ID: ck1p3hu7fwdvv0748af2t1o9q
Image: Incendio_Ester_20190602153400_20190602155200_20_F_Main_mp4_386, ID: ck2iq69e0uqi70794hbrl6r1d
Image: 2019-01-30 10-43-49_NAO.jpeg, ID: ck27h5chpfx0l0811diz9arn5
Image: 2018-03-08 

Image: 2018-03-04 19-26-53_NAO.jpeg, ID: ck1p42g394ry40944e6xawhtq
Image: 2018-03-17 12-00-53_SIM.jpeg, ID: ck1wqlmb8mh740721m1svdrhn
Image: 30_R_19082018_142912_mp4_0, ID: ck2jjfpbvxz970794aorre2tp
Image: 2018-03-04 07-22-23_NAO.jpeg, ID: ck1to07nj60gp083824ycbfrm
Image: 2018-03-04 09-33-57_NAO.jpeg, ID: ck1v9nrvvxgd90944yuarscuv
Image: 2018-03-14 15-43-52_NAO.jpeg, ID: ck2ma3r5pygb70944ypdklyg6
Image: 2018-03-02 11-58-48_NAO.jpeg, ID: ck2jd3q34zf590838831a11ra
Image: 2018-02-27 11-17-04_SIM.jpeg, ID: ck225lhkbx6tj07488sekwg37
Image: 2019-09-15 07-20-20_NAO.jpeg, ID: ck25nchtb8si20794ouitky44
Image: 2018-03-02 09-38-47_NAO.jpeg, ID: ck1wqcbga0jww0811yt1yutnl
Image: 2018-03-06 20-48-52_NAO.jpeg, ID: ck1yvybrqwxok072114aqjz3h
Image: 2018-03-15 17-45-28_NAO.jpeg, ID: ck1tfyjuu928m0721yeam217x
Image: 2018-03-20 08-04-30_NAO.jpeg, ID: ck227neh2zeuy0794p8wezp3q
Image: 2018-08-02 10-07-56_NAO.jpeg, ID: ck1nzhl44vsc208381wbi08uo
Image: 2018-03-20 10-29-26_NAO.jpeg, ID: ck29a8yep4a2k0838hqedkc

...

Image: 2018-11-04 07-45-40_NAO.jpeg, ID: ck29d83nc2koe07944ucefxr1
Image: 2019-02-01 20-05-15_NAO.jpeg, ID: ck29d95xg7wn50944j5w7hn98
Image: Incendio_IP_30_R_19082018_142912_mp4_2205, ID: ck2ir7vlzgjtz0811v79fqg8n
Image: Incendio_IP_30_R_19082018_142912_mp4_401, ID: ck2jmyijr57kr0721545lfy2m
Image: 2018-03-02 15-38-29_NAO.jpeg, ID: ck1o9r99y1tdt0944rxzgb9lz
Image: 2018-03-18 09-50-34_NAO.jpeg, ID: ck29dc5ba4pev0838z7obj51b
Image: Incendio_IP_IP_30_R_19082018_142912_mp4_201, ID: ck2iqegnuwzpx0838726mbmh5
Image: Incendio_IP_30_R_19082018_142912_mp4_1303, ID: ck2ipqbtuzidf0944t7bv5038
Image: 2018-03-18 12-31-41_NAO.jpeg, ID: ck28kdj016zpe0721f2ztf00p
Image: 2018-03-18 08-47-19_NAO.jpeg, ID: ck28mptyn88i9075706ejfsuj
Image: 2018-03-13 16-43-58_NAO.jpeg, ID: ck1p3axow5u6807212fz7r113
Image: 2018-03-14 06-35-20_NAO.jpeg, ID: ck1pfbb05xrpj0748k3igmif8
Image: 2018-02-26 15-59-46_NAO.jpeg, ID: ck1w3itnqr91708110bvneph4
Image: 2018-03-14 10-30-47_NAO.jpeg, ID: ck1v3zqfuvhpi0944uls2czrw
Image: 20

Image: 2018-03-16 10-36-56_NAO.jpeg, ID: ck1wo3sqp6kcl0725i2r3o5ib
Image: Incendio_Ester_Ester_5_mp4_194, ID: ck2jmf61j555z072162hw2akq
Image: 2018-03-16 10-45-18_NAO.jpeg, ID: ck1yegt4pfu6f0725aizl4tdl
Image: 2018-03-14 10-54-12_NAO.jpeg, ID: ck1owc7n03tw10944ow1ayk67
Image: 2018-03-02 10-30-06_NAO.jpeg, ID: ck2enp1z0wat00794ny224use
Image: 2018-03-14 16-23-22_NAO.jpeg, ID: ck1zrdsyorawc07489f8juek6
Image: 2018-03-02 05-30-18_NAO.jpeg, ID: ck2j39aj3hk610811tnck68j8
Image: 2017-12-12 12-58-24_SIM.jpeg, ID: ck1w4vjvbe22y07215etdsx59
Image: 2019-01-15 08-43-40_SIM.jpeg, ID: ck29b73srnmfi0811dzpbmcnz
Image: Outras_sem_incendio_mp4_55, ID: ck2i90jarvwej08384ib2w4rv
Image: 2018-03-16 12-57-18_NAO.jpeg, ID: ck2jbgkgv3t220721tgg37j5x
Image: 2018-03-02 05-35-07_NAO.jpeg, ID: ck1wel0nijzym0757otdpw8vk
Image: 2018-03-03 23-41-56_NAO.jpeg, ID: ck1s6f7i2kvbr0944znwyghrd
Image: 2018-03-13 17-24-18_NAO.jpeg, ID: ck1qdbzeve3if0748v5fkw2p9
Image: Outras_Noturno_mp4_770, ID: ck2jcbrzvva8g0748cw1gwjrv
I

Image: 2018-03-03 11-45-21_NAO.jpeg, ID: ck1qldr8owfyc0757tm73i1yg
Image: 2019-08-03 19-03-18_SIM.jpeg, ID: ck23mhxpugd520748ickta8fu
Image: Incendio_IP_10_R_17072018_103000_mp4_379, ID: ck2i80ezt2nw3075784268rcp
Image: 2018-02-05 16-28-52_SIM.jpeg, ID: ck1w7mg599te508384wqmgwrn
Image: 2019-07-26 09-44-24_SIM.jpeg, ID: ck27qm84r1rkc0944m7mpznym
Image: 2018-11-04 07-33-47_NAO.jpeg, ID: ck27vjq02x6200794t92emym8
Image: 2018-12-03 19-20-13_NAO.jpeg, ID: ck2n2shb79k6r09444yizvgjb
Image: 2018-03-19 10-17-43_NAO.jpeg, ID: ck227nwtgxway0748nx049cxv
Image: 2018-03-16 08-13-48_NAO.jpeg, ID: ck223vgyejh7v0811fmuldzst
Image: Incendio_TTG_-_TUR_1_mp4_266, ID: ck2iao8vukma307251qjg9owj
Image: 2018-03-20 16-48-49_NAO.jpeg, ID: ck27ep0ytw5n508386mrbycmc
Image: 2018-03-08 06-53-13_SIM.jpeg, ID: ck1oepmumh5s808118y70jw0z
Image: Incendio_IP_10_R_17072018_103000_mp4_652, ID: ck2iqxkxh1p210721mfczbq24
Image: 2018-03-16 09-19-23_NAO.jpeg, ID: ck1own52cvjbw0748oz2ffpn6
Image: 30_R_19082018_142912_mp4_35, ID

Image: 2018-03-16 17-20-23_NAO.jpeg, ID: ck20nlubwbdli07486zrj6jz0
Image: 2018-03-21 16-48-20_SIM.jpeg, ID: ck29auevf9w020721ylg2fppi
Image: Incendio_10_R_17072018_103000_mp4_1008, ID: ck2j7fz5yylzg08387csevj96
Image: 2018-10-27 07-57-34_SIM.jpeg, ID: ck2n16ehdul3l0725ter1bgbq
Image: 2019-04-01 19-13-09_NAO.jpeg, ID: ck23y57yojp6c07489a22f5gj
Image: 2018-03-21 13-13-43_SIM.jpeg, ID: ck23zaw2mu2040757cw8e0o1e
Image: 2018-03-20 15-22-57_NAO.jpeg, ID: ck24n1wxj2inp07217sjsxeno
Image: 2018-01-31 14-01-14_NAO.jpeg, ID: ck2ep0o7n40ky0721qvktrhyi
Image: 2018-03-18 08-13-14_NAO.jpeg, ID: ck2370cjtgij10944r0i7akql
Image: Incendio_10_R_17072018_103000_mp4_1195, ID: ck2ir8y343yd20757satwduan
Image: 2018-03-16 11-49-57_NAO.jpeg, ID: ck1to7hz8pqdo0811s0v1jgnk
Image: 2018-03-16 15-06-54_NAO.jpeg, ID: ck1tpru374nbi0794cz890cxb
Image: 2018-03-14 07-01-37_NAO.jpeg, ID: ck1wr98r5js340944poba3psa
Image: 2018-03-12 16-31-42_NAO.jpeg, ID: ck1zv7lhj1gk00721jdn48t0i
Image: 2018-03-17 21-43-55_SIM.jpeg, ID: c

Image: Incendio_Amcel_4_Torre1_mp4_0, ID: ck2j7f1th39h90721n27rtpie
Image: 2018-03-15 17-25-09_NAO.jpeg, ID: ck2hw8xd41bvf0757yage1o48
Image: 2018-03-17 11-08-09_SIM.jpeg, ID: ck1rkutnj71vb09446fjd2bo2
Image: 2018-03-02 12-08-40_NAO.jpeg, ID: ck2j7kqud5iuy0757am8tmgrh
Image: 2018-03-13 17-09-25_SIM.jpeg, ID: ck1wqm2mwgsyf0838hhfg7iyt
Image: 2018-01-30 09-13-51_SIM.jpeg, ID: ck1vlen0zxvmt0838lnvf1avy
Image: 2019-02-07 15-27-57_NAO.jpeg, ID: ck29c9o6fnrvb0811nbueyity
Image: 2018-03-14 06-41-09_NAO.jpeg, ID: ck2mcyeaogtbp0811olqbayfd
Image: 2019-07-30 09-37-03_SIM.jpeg, ID: ck281v8ba5h2w072102aaxxfb
Image: 2018-01-30 11-22-32_NAO.jpeg, ID: ck20oy0pudqv90794g8b953yw
Image: 2018-03-05 06-12-07_NAO.jpeg, ID: ck1p45xu75xgx0721dtfviq25
Image: 2018-03-11 22-40-56_SIM.jpeg, ID: ck1t5z2r6idc70811rpq1zih4
Image: 2018-03-04 08-46-46_NAO.jpeg, ID: ck1wv1cc6fje50794zx6yzo4e
Image: Incendio_Ester_20190602153400_20190602155200_20_F_Main_mp4_194, ID: ck2ir2ij7x1wm08383ojrcr40
Image: Incendio_IP_30_R_190

In [73]:
load_labelbox(df_test, test_output_dir)

Image: 2018-03-14 06-54-11_NAO.jpeg, ID: ck2j7qooln89s07255dqax60y
Image: 2018-03-20 10-24-33_NAO.jpeg, ID: ck2n2l75s4qhv0794udy4jf5w
Image: 2018-03-19 13-09-47_NAO.jpeg, ID: ck27wam0mvqfh0748nn9ie1l1
Image: 2019-09-23 07-28-24_SIM.jpeg, ID: ck27vgog05oaz0757ufrzbqtg
Image: 2018-03-14 06-25-33_NAO.jpeg, ID: ck1z3lvhdxicz07210zak5rln
Image: 2018-03-19 11-50-08_NAO.jpeg, ID: ck2931rcu3es60838dpxf9k7f
Image: 2018-02-24 08-43-37_NAO.jpeg, ID: ck20ibsc2a5f00794h1rg7e9g
Image: 2018-03-04 11-51-16_NAO.jpeg, ID: ck1urituqnx1q0838mqktcq7x
Image: 2018-03-12 14-01-42_NAO.jpeg, ID: ck1xq8fn0p3jm0944yxx1iurw
Image: 2018-03-18 08-46-40_NAO.jpeg, ID: ck29e2kniu1o307258jhhu4ij
Image: 2018-03-12 09-29-20_NAO.jpeg, ID: ck1ql21wksu6f0944e9ey13q0
Image: 2019-01-30 10-45-44_NAO.jpeg, ID: ck23o11qak5iq0838x4l43l3u
Image: 2018-03-09 21-16-14_SIM.jpeg, ID: ck2jba8fswvfr0794kzphhr2g
Image: 2018-03-14 07-12-49_NAO.jpeg, ID: ck1wrbzazdn6k07484g0jj26f
Image: 2018-03-14 08-31-03_NAO.jpeg, ID: ck1uoc7zfkhft0794iffr

Image: 2018-03-13 15-58-04_NAO.jpeg, ID: ck20u5hdcop3o0757fpwi8y7v
Image: Incendio_IP_30_R_19082018_142912_mp4_427, ID: ck2jcaz991tqq0944a85j1j3e
Image: Incendio_TTG_-_TUR_1_mp4_94, ID: ck2ipttxt3tz20757crerroew
Image: 2018-03-01 23-21-36_SIM.jpeg, ID: ck1qd4o59dzcx07482fhdn06w
Image: 2018-11-02 18-44-53_NAO.jpeg, ID: ck28dd1di7ipj0757qmkt8b33
Image: 2019-02-14 09-02-09_SIM.jpeg, ID: ck27el933fcax0811zmke51ga
Image: 2018-03-16 12-55-39_NAO.jpeg, ID: ck1wckxc9f15p0944ikxy2xq3
Image: 2018-03-10 10-49-21_NAO.jpeg, ID: ck2mck4ojgodk0811di9rbohv
Image: 2018-03-02 10-16-11_NAO.jpeg, ID: ck1t0knm10pfj0721k31h7z5m
Image: 2018-03-02 12-17-58_NAO.jpeg, ID: ck1p1fvikj6nr081184gm9ch4
Image: 2018-11-04 07-46-14_NAO.jpeg, ID: ck25dfq1fdhjf0721066h41qk
Image: Incendio_Ester_20190602153400_20190602155200_20_F_Main_mp4_217, ID: ck2j8or4ywk3o0794cdt8l26a
Image: 2018-03-20 12-06-13_NAO.jpeg, ID: ck29b9q3e4fbk0838w2rp7fry
Image: 2018-03-16 08-00-02_NAO.jpeg, ID: ck1zynrafu4pz0748vtydk1dx
Image: 2018-03-16

Image: 2018-03-15 13-56-48_SIM.jpeg, ID: ck1z2ocpkpzjh07941mba2af6
Image: 2018-03-14 08-35-18_NAO.jpeg, ID: ck1vgfzhbyrc00944e4pbcmez
Image: 2018-03-16 07-30-58_SIM.jpeg, ID: ck1w6do10c2nz0944odkc7j3q
Image: Incendio_IP_30_R_19082018_142912_mp4_1224, ID: ck2jbrc3pz6u50838qctavjdn
Image: 2018-03-06 05-56-39_SIM.jpeg, ID: ck1u5l76tcrqy0838f4zer94s
Image: 2018-03-20 15-08-53_NAO.jpeg, ID: ck23x92k3l1yt07941igebz28
Image: 2018-03-01 23-07-12_SIM.jpeg, ID: ck1wrqbl6jun60944xsrc7cmc
Image: 2018-03-02 10-04-31_NAO.jpeg, ID: ck20r1vh0ngjq07574bku5ivt
Image: 2019-09-23 07-26-55_SIM.jpeg, ID: ck29a4z4mngv80811egjpuw86
Image: 2018-03-04 06-18-50_SIM.jpeg, ID: ck20tr1bzfy3p0794bm8dka32
Image: 2018-03-19 22-00-39_SIM.jpeg, ID: ck27y0ru350ty07219ovmndvq
Image: 2018-03-19 19-33-22_SIM.jpeg, ID: ck2n2diem9a8o0944r7188z4q
Image: 2019-01-30 11-11-18_NAO.jpeg, ID: ck25ogg5tholc0757dhym26e4
Image: 2018-03-09 21-57-47_SIM.jpeg, ID: ck1uh0imogu2c0794qv7jtd4k
Image: 2018-03-04 12-48-08_NAO.jpeg, ID: ck1p6isn

Image: 2018-03-20 16-57-58_NAO.jpeg, ID: ck2ml1qnlnmtt0725rbh9c9ta
Image: 2018-03-03 20-57-25_NAO.jpeg, ID: ck1qlkdo8kvkd0748dwowe46f
Image: Outras_sem_incendio_mp4_612, ID: ck2in3ypmg8za08113qbqptye
Image: 2018-03-19 09-01-30_SIM.jpeg, ID: ck28290p4399t09442g2s45jq
Image: 2018-03-20 17-38-58_NAO.jpeg, ID: ck22t42flqbi70811gadmysba
Image: 2018-03-08 07-53-41_NAO.jpeg, ID: ck2m7ny2jzl8d07213dbpt7ly
Image: Incendio_IP_IP_30_R_19082018_142912_mp4_37, ID: ck2jmtien31uy0944z2gqa6vs
Image: 2019-05-19 08-44-56_SIM.jpeg, ID: ck256c8dq89xv0944icxz5xb0
Image: 2018-03-15 15-32-03_NAO.jpeg, ID: ck1w7ri4bcsys0944no0j9yh5
Image: 2018-03-04 15-54-56_NAO.jpeg, ID: ck1owwq6wxvo40794t2uoegjf
Image: Outras_sem_incendio_mp4_809, ID: ck2jn2a89p72v0725bdlk2f44
Image: 2018-03-16 13-06-22_NAO.jpeg, ID: ck2mdbpg4x42i0838qwh1i01m
Image: 2018-03-18 09-15-50_NAO.jpeg, ID: ck2eorikiv0ai0748488owk76
Image: 2018-03-19 22-46-43_SIM.jpeg, ID: ck23mptk236u90811lfwtc3nf
Image: Incendio_IP_IP_30_R_19082018_142912_mp4_343

Image: 2018-01-09 15-42-26_NAO.jpeg, ID: ck1whhqm2aivx0748se521fkq
Image: 2018-01-09 16-26-53_NAO.jpeg, ID: ck1x99thdjliq08388lj1d7j4
Image: 2018-03-02 09-27-22_NAO.jpeg, ID: ck2me1mzhv1v70794vehhm7dy
Image: 2018-02-14 14-56-43_NAO.jpeg, ID: ck2mepkp6m5hp0725r3s5mpa5
Image: Outras_sem_incendio_mp4_1384, ID: ck2iqojl9llar0725o6vsgwm8
Image: 2018-03-20 14-11-59_NAO.jpeg, ID: ck2mzt36c35870794t7tz4idm
Image: 2018-03-20 17-24-52_NAO.jpeg, ID: ck23y6z0bn0or0838nb1j4835
Image: 2018-01-30 16-00-51_SIM.jpeg, ID: ck1s3wb9wedyz0794nltbxotz
Image: 2018-03-15 17-13-27_NAO.jpeg, ID: ck20tei9gofhm07572lwjljp7
Image: 2018-03-13 15-55-07_NAO.jpeg, ID: ck1w6cz6t7dzx0794aaf9hy4f
Image: 2018-03-17 13-05-29_NAO.jpeg, ID: ck20ol1lkyv7j0811fg5yrbc4
Image: 2018-03-18 08-09-19_NAO.jpeg, ID: ck26bbyljl36o083803a4k3yw
Image: 2018-03-14 06-13-41_NAO.jpeg, ID: ck1w80q1f0hec0725z709349y
Image: 2018-03-13 16-48-56_NAO.jpeg, ID: ck1qd50z8pmlc0757omncsp5f
Image: 2018-03-14 16-06-58_NAO.jpeg, ID: ck1xffui7rsh207575912

...

Image: 2019-03-14 08-46-22_SIM.jpeg, ID: ck23cgo6nfroa0838jcobj9g9
Image: 2018-03-15 17-41-44_NAO.jpeg, ID: ck1w86j6v0ksf07251q01ou92
Image: 2018-03-10 10-43-52_NAO.jpeg, ID: ck20o6nb4kddf07218zt86uzd
Image: 2018-03-20 14-03-28_NAO.jpeg, ID: ck2mwusivswz00725hi7dkz6f
Image: 2018-03-20 15-39-26_NAO.jpeg, ID: ck28axuno447n0944w8rqbkhl
Image: 2018-03-08 08-09-51_NAO.jpeg, ID: ck1uqsrlanlaa0838gjnu3smg
Image: 2018-03-02 15-03-00_NAO.jpeg, ID: ck1ooveq7yhnf083885nehrdq
Image: 2018-03-12 08-16-40_NAO.jpeg, ID: ck1zsonu7xqpq0944kjyb9lmi
Image: 2018-03-09 21-17-26_SIM.jpeg, ID: ck208xbcl222v07941x0g1z63
Image: 2018-03-02 00-02-04_NAO.jpeg, ID: ck1wq0blbex6w0794vxvr5ynx
Image: 2018-03-04 11-22-40_NAO.jpeg, ID: ck1qpewb2qny80838kpyawpmb
Image: 2018-03-13 21-32-01_SIM.jpeg, ID: ck2i631ph2gc307577iame3x6
Image: 2019-09-11 07-14-31_NAO.jpeg, ID: ck237vhcakp0r0757smzidca9
Image: Incendio_Ester_Ester_4_mp4_196, ID: ck2iakuey2wdq075716anerkl
Image: 2018-03-08 09-55-00_NAO.jpeg, ID: ck1qpb5k9vw7607210q

Image: 2018-03-17 13-29-57_NAO.jpeg, ID: ck1woh2ji01p10811j4y20md1
Image: 2018-03-12 16-08-51_NAO.jpeg, ID: ck1ovs2u7q2jo07257xbxwpr1
Image: 2018-03-08 15-49-42_NAO.jpeg, ID: ck1sshrrcsrad0944aokr28db
Image: 2018-03-15 23-17-17_NAO.jpeg, ID: ck1wps3jhnnxw0757wj35z99k
Image: 2018-03-15 19-14-42_NAO.jpeg, ID: ck1w60jqgzaoc072546fps98v
Image: 2018-03-08 20-48-43_NAO.jpeg, ID: ck20ntcssk77v07215i4irzab
Image: 2018-03-18 08-46-14_NAO.jpeg, ID: ck28mg4gnqznr07257plcafuw
Image: 2018-03-15 23-26-08_SIM.jpeg, ID: ck1xqghpjrvh60721qbmagk85
Image: 2018-03-16 16-19-06_NAO.jpeg, ID: ck2mev5trhg100811xeqd9qc4
Image: Incendio_IP_30_R_19082018_142912_mp4_1823, ID: ck2j11w1m0f6f0944ie0m1qos
Image: Outras_sem_incendio_mp4_1361, ID: ck2iqpgumurvi07945y8n8ke9
Image: 2018-01-09 15-31-06_SIM.jpeg, ID: ck1s3kyl7chcc07483eq97quc
Image: 2018-03-20 15-08-23_NAO.jpeg, ID: ck27hh9ul00m20944jokew425
Image: 2018-03-04 10-14-56_NAO.jpeg, ID: ck1uqu04wdxtz072565ccuyt8
Image: 2018-03-16 15-32-30_NAO.jpeg, ID: ck2mci44

Image: 2018-02-23 17-23-39_NAO.jpeg, ID: ck2j62jeo0zb80944qhih7uvg
Image: 2018-03-16 10-35-41_NAO.jpeg, ID: ck224ox57jrx508118snz1ghr
Image: 2018-03-19 10-57-33_SIM.jpeg, ID: ck27ro0kd1z8h094459rtd8l3
Image: 2018-03-14 10-38-03_NAO.jpeg, ID: ck1rwlg7tvi500811mp34q3bz
Image: 2018-03-15 19-47-27_SIM.jpeg, ID: ck1rkfrbd0uf80794vujef2yu
Image: 2018-03-04 19-17-42_NAO.jpeg, ID: ck1wqh7famg6d0721ufdi5z7q
Image: 2018-03-02 10-40-56_NAO.jpeg, ID: ck1vkxb75ua8l07489f450n7w
Image: 2018-03-06 09-57-23_NAO.jpeg, ID: ck1rwg1k52qsu0725gchvvc22
Image: 30_R_19082018_142912_mp4_679, ID: ck2iqdhwylkk70725ou0ceryf
Image: 2018-03-17 09-44-04_NAO.jpeg, ID: ck1qk681ovsyf0757z823r6kp
Image: 2018-03-16 09-14-36_NAO.jpeg, ID: ck1x2a03jgkc50794vo5jfywx
Image: 2018-03-02 12-45-37_NAO.jpeg, ID: ck1wijgi6jv3t0721sk53bind
Image: 2018-03-17 13-01-26_NAO.jpeg, ID: ck1unnt48ikdb07489ucbr7mx
Image: 2019-01-30 21-42-48_NAO.jpeg, ID: ck23c93y7irj90944xf7rr6ja
Image: 2018-03-14 08-32-15_NAO.jpeg, ID: ck1p4wkwfr7fh0725qits

Image: 2018-03-19 20-43-01_SIM.jpeg, ID: ck2972l7k0boo0748is3lmbjj
Image: 2019-09-14 18-18-07_NAO.jpeg, ID: ck2531odc1wd807940qr9km3j
Image: 2019-02-01 20-05-45_NAO.jpeg, ID: ck28gciwxkhrq0811tr0nht6h
Image: 2018-03-11 22-38-08_NAO.jpeg, ID: ck2mcia3xlept07252k8hqosc
Image: 2018-03-15 20-44-28_SIM.jpeg, ID: ck1yw8bvwpj460794e2etn4q4
Image: 2018-01-09 15-34-22_SIM.jpeg, ID: ck20uydv3i8zf0838bidmoe2f
Image: 2018-03-06 09-12-07_NAO.jpeg, ID: ck1s6cr338e6g07253trsq7kh
Image: 2019-02-18 15-38-21_SIM.jpeg, ID: ck255zat839pd0794iy6s2zh0
Image: 2018-03-18 17-45-30_NAO.jpeg, ID: ck22b0brb98in0757t1gyb802
Image: 2018-01-31 14-11-39_NAO.jpeg, ID: ck1ql1mo4wa7n07576fikf1n9
Image: Outras_transicao_dia_noite_mp4_429, ID: ck2irkhd4uvjg0794pkr37p9w
Image: 2018-03-20 09-27-26_NAO.jpeg, ID: ck23rb2prhxps0748i5spe28u
Image: 2018-03-17 15-23-47_SIM.jpeg, ID: ck1qljbl0kv4s0748fx2f7unp
Image: Incendio_TTG_-_TUR_1_mp4_208, ID: ck2is5zr3x65z08380teqjrk7
Image: 30_R_19082018_142912_mp4_2018, ID: ck2iry1v3zpin0

Image: 2018-03-18 12-09-39_NAO.jpeg, ID: ck27wcz3czd0c0838o2rfduix
Image: 2018-03-04 10-07-52_NAO.jpeg, ID: ck1ogwhdz2jkt0944nziospzg
Image: 2018-03-10 10-38-33_NAO.jpeg, ID: ck1xg8jozhm370748h7b8twex
Image: 2018-03-20 06-29-26_NAO.jpeg, ID: ck2n2qen49iwe0944jfzlf2we
Image: 2018-03-02 19-16-41_NAO.jpeg, ID: ck1xhwkc9bmce0725bwwyu46i
Image: 2018-03-16 13-11-34_NAO.jpeg, ID: ck1v9m72puizh08380v4egqaf
Image: 2018-03-04 12-32-08_NAO.jpeg, ID: ck1wqo5vimhjb0721kof6dlek
Image: 2018-03-18 11-48-03_NAO.jpeg, ID: ck2n1apqd26hx0748d3hpm86g
Image: 2018-03-18 17-09-06_NAO.jpeg, ID: ck23842cbxfw40811r1r6bt5q
Image: Incendio_Amcel_2_Torre5_mp4_0, ID: ck2i9r5xytqmo0794bwrvft4e
Image: 2018-03-13 17-36-10_NAO.jpeg, ID: ck1w6h5gy5xzf07489rtf9in2
Image: 2018-03-17 13-49-36_NAO.jpeg, ID: ck1z36vnfble30811ejirfn5j
Image: 2018-03-19 16-37-32_NAO.jpeg, ID: ck27wwdcivt0r0748dha9yw3q
Image: 2018-03-10 06-10-38_SIM.jpeg, ID: ck1vl1ynt4vah07573iv10f7d
Image: Incendio_IP_10_R_17072018_103000_mp4_624, ID: ck2i8yys

In [77]:
load_labelbox(df_train, train_output_dir)

Image: 2018-03-18 13-08-56_NAO.jpeg, ID: ck26qatq5qxtz08383lt9k9km
Image: 2018-03-14 06-15-45_NAO.jpeg, ID: ck1wow2alemv40794ubazlrl3
Image: 2018-03-13 19-48-01_NAO.jpeg, ID: ck2jb9u393s9p07214zlg3ors
Image: 2018-03-15 15-08-11_SIM.jpeg, ID: ck1p46cohr3ld0725del4re3l
Image: 2018-03-02 19-27-52_NAO.jpeg, ID: ck2mb47kl0uzs0721zciluur8
Image: 2018-03-12 10-14-31_NAO.jpeg, ID: ck1w5icvaefrt072127y87ql3
Image: Incendio_IP_30_R_19082018_142912_mp4_271, ID: ck2i5djhyy2420944ns5rn6ps
Image: Incendio_IP_30_R_19082018_142912_mp4_300, ID: ck2j8epaonc0u0725vzvlzfvk
Image: 2018-03-19 23-16-06_NAO.jpeg, ID: ck29dqv3jaab10721c36dksew
Image: 2019-03-14 11-54-58_SIM.jpeg, ID: ck27er3fv2n7k07574xc2egqu
Image: 2018-02-26 18-09-02_NAO.jpeg, ID: ck1v3o0kmitj00725senjypj6
Image: 2018-02-26 17-06-56_NAO.jpeg, ID: ck1uoadn6kgd90794ylefz5d5
Image: 2018-03-16 12-44-11_NAO.jpeg, ID: ck2mh1shxmv0z0725bm8zqyez
Image: 2018-03-09 22-07-42_SIM.jpeg, ID: ck1p3nzztwefi0748ux3q2apc
Image: Incendio_IP_30_R_19082018_14291

Image: 2018-03-12 17-42-54_NAO.jpeg, ID: ck1rw6ofpfl0m0944bsulto1w
Image: 2018-03-04 08-04-04_NAO.jpeg, ID: ck1whgn3caika0748nd7hncrc
Image: 2018-03-16 14-41-03_NAO.jpeg, ID: ck1p3f4pdypmt079494qiugpa
Image: 2018-03-18 16-58-23_NAO.jpeg, ID: ck22awhtqsgj70725pjixm9ib
Image: Outras_sem_incendio_mp4_854, ID: ck2j6his95e1q075760fjcy7r
Image: 2018-03-16 11-26-13_NAO.jpeg, ID: ck1zs80wp1pih07577q581ar9
Image: 2018-03-04 12-18-57_NAO.jpeg, ID: ck1rz8ro8lobd0757ij6cc4m2
Image: 2018-03-08 07-47-09_SIM.jpeg, ID: ck1w8imu7d9fp0944cjr49lh3
Image: 2018-03-15 18-15-44_NAO.jpeg, ID: ck20w63lhpfgc0757dxm0bltl
Image: 2018-01-10 12-10-17_SIM.jpeg, ID: ck1yz51pwbbfu0811xd8c9mr3
Image: 2018-03-18 14-39-32_NAO.jpeg, ID: ck29asl85nkhr0811pybs8q33
Image: Incendio_IP_10_R_17072018_103000_mp4_733, ID: ck2jj7phc72130757sghqos43
Image: 2018-03-17 09-47-03_NAO.jpeg, ID: ck1vlar06uhu00748ppknhlnu
Image: 2018-02-24 08-40-42_NAO.jpeg, ID: ck1z3lo64i9gq0725ojrja35p
Image: 2018-03-12 17-46-51_NAO.jpeg, ID: ck20uixq7g

Image: 2018-03-12 13-26-07_NAO.jpeg, ID: ck1w3j2uhr95x0811bnprv0nu
Image: 2018-03-16 08-32-53_NAO.jpeg, ID: ck1tesi0o2u6t0838kijlaxoz
Image: 2018-03-04 08-22-26_NAO.jpeg, ID: ck1p2yfpcjfj30811xogd2z5w
Image: 2018-03-20 14-23-26_NAO.jpeg, ID: ck29d9ldy2ktv079426fox1cp
Image: 2018-03-21 14-02-28_NAO.jpeg, ID: ck29359x5mmc008117z7sro0d
Image: 2018-02-26 11-34-34_NAO.jpeg, ID: ck1w80hnntu4z08114r1y6akz
Image: 2018-03-18 08-25-23_NAO.jpeg, ID: ck29dephx4pu508380v7un9oo
Image: 2018-03-14 13-26-52_NAO.jpeg, ID: ck1yxxsg7husk0725ngpdk5re
Image: 2018-03-13 18-31-17_NAO.jpeg, ID: ck2jb2mpiwumf0794zb7srxzr
Image: 2018-03-20 07-53-26_NAO.jpeg, ID: ck29btr12b1g70757mh6synb1
Image: 2018-03-14 06-51-50_NAO.jpeg, ID: ck1rznsd5ly3w0757zcit199h
Image: 2018-03-20 09-29-44_NAO.jpeg, ID: ck28edelaqdyp072560j7t4b5
Image: 2019-06-28 18-21-12_NAO.jpeg, ID: ck26cat8yaylr07250toaec74
Image: 2018-03-18 16-36-39_NAO.jpeg, ID: ck29au8dqnkq80811daxe64xm
Image: 2018-03-16 10-56-10_NAO.jpeg, ID: ck1rksjlczg2m07483z8g

Image: 2018-03-12 09-15-00_NAO.jpeg, ID: ck1wop9utm07y0721cid9m13r
Image: 2018-02-15 10-42-58_SIM.jpeg, ID: ck2mezv8j4qls0757ulbzz3c5
Image: 2018-03-04 11-01-07_NAO.jpeg, ID: ck1tpk0qo35lj0748pc977if0
Image: 2018-03-17 19-54-49_SIM.jpeg, ID: ck26auweche1t0748quonfa2e
Image: Incendio_Ester_Ester_5_mp4_99, ID: ck2jbmv1z1qmd0944ww48952z
Image: 2019-01-30 10-15-10_NAO.jpeg, ID: ck25oamezhm2v0757r334c0o1
Image: 2018-03-12 09-17-17_NAO.jpeg, ID: ck2j8b8omwhoc0794kxdc5g2h
Image: 2018-03-19 08-43-01_NAO.jpeg, ID: ck23bgg1il1hl0721ji3rfte5
Image: Incendio_10_R_17072018_103000_mp4_682, ID: ck2jjeetw2pf70944jbzefyjt
Image: 2019-03-14 08-36-11_SIM.jpeg, ID: ck2mzy5645esq08387snuu06e
Image: 2018-03-18 07-52-06_NAO.jpeg, ID: ck2n089lgu3l30725rmk8wmbe
Image: Incendio_IP_IP_30_R_19082018_142912_mp4_1081, ID: ck2in7ldxss2f074893mh9xec
Image: 2018-03-13 19-11-32_NAO.jpeg, ID: ck20qsoswd2mp0748h1hso8wb
Image: 2018-03-20 06-45-19_NAO.jpeg, ID: ck28lwmb0kvxn0811ktbu3nx8
Image: 2018-03-13 15-48-12_NAO.jpeg,

Image: 2018-03-20 10-33-22_NAO.jpeg, ID: ck24l5jgj2z1e0757oehzuwru
Image: 2018-03-20 10-38-24_NAO.jpeg, ID: ck283079y3b700944ukf14ge6
Image: 2018-03-07 21-08-23_SIM.jpeg, ID: ck1oeaf8bomyo0725jalfq2rr
Image: 2018-03-02 05-21-51_NAO.jpeg, ID: ck1wrtbusf8us0794a6qwg523
Image: 2018-03-12 17-20-42_NAO.jpeg, ID: ck1p3xslfwfbb0748pbl7rl0n
Image: 2018-03-17 11-17-55_NAO.jpeg, ID: ck20snt4wfjvr0794a3eale2n
Image: 2018-03-18 10-06-06_NAO.jpeg, ID: ck23dlnzan9460757ek4wge7t
Image: Incendio_Ester_20190602153400_20190602155200_20_F_Main_mp4_486, ID: ck2j5h2th0w8q0944146crs9u
Image: 2018-11-11 22-14-33_NAO.jpeg, ID: ck2807dve5alx0721syq3rlbr
Image: 2018-03-14 12-56-32_NAO.jpeg, ID: ck20nqq7yd2ge07947trd2qpc
Image: 2018-11-14 05-58-41_NAO.jpeg, ID: ck255dlnlum8b0725ckv62tco
Image: 2018-03-18 09-04-05_NAO.jpeg, ID: ck27s7til59pk0757rw29ir8o
Image: 2018-02-27 15-04-38_NAO.jpeg, ID: ck1w6a7gag9810757ha36it2n
Image: 2018-03-03 20-43-29_NAO.jpeg, ID: ck22hun4c9hw407213gav6o32
Image: 2018-03-04 10-35-46_N

...

Image: 2019-01-30 10-53-49_NAO.jpeg, ID: ck27s810z59qm07579b96jb7o
Image: 2018-03-14 07-08-38_NAO.jpeg, ID: ck1v3hs5uirze0725tcawamxs
Image: 2018-03-01 21-51-02_NAO.jpeg, ID: ck1z4tnmsrv640838w9ilpjc9
Image: 2018-03-17 10-59-21_SIM.jpeg, ID: ck1utn3f0oxe30838b0mrbo8o
Image: 2018-03-16 09-36-54_NAO.jpeg, ID: ck1wns45mln790721r1t4m00l
Image: 2018-03-18 09-25-39_NAO.jpeg, ID: ck23xz0fj6h3h0811p3m04edf
Image: 2018-03-18 09-27-29_NAO.jpeg, ID: ck27w3uzl5rbr0757ufohe0ug
Image: 2018-03-15 19-07-28_SIM.jpeg, ID: ck1yvuashr7ja0838jr5814tr
Image: 2018-03-16 12-31-08_NAO.jpeg, ID: ck1tt13ixrtw20811iooz98rz
Image: 2018-03-04 14-55-43_NAO.jpeg, ID: ck1p3ptcqyqul07943d87abkv
Image: 2018-03-16 15-07-56_NAO.jpeg, ID: ck1ugtujdlb7u09448y75gg0w
Image: 2018-03-14 16-17-49_NAO.jpeg, ID: ck1y6gbztlg6j07488qpozgzf
Image: 2018-03-08 12-56-16_NAO.jpeg, ID: ck1p5lpn3jrtd0811p7sy3quy
Image: 2019-07-22 10-01-23_SIM.jpeg, ID: ck22sufq1dlwg0757akg5ets2
Image: 2018-03-02 10-28-19_NAO.jpeg, ID: ck1owq4ekq5n707258ymu

Image: 2018-03-10 07-15-38_SIM.jpeg, ID: ck1w51v6954wy07489m86t2d9
Image: 2018-03-20 09-28-52_NAO.jpeg, ID: ck251wqy51efr0794jshdearc
Image: Incendio_Ester_20190602153400_20190602155200_20_F_Main_mp4_805, ID: ck2j8m3uzwjrn0794z4k3aymk
Image: 2018-03-02 09-00-01_NAO.jpeg, ID: ck1oguo38wjy80794qs0p76vf
Image: 2018-03-06 09-49-39_NAO.jpeg, ID: ck1w3enq8d25u0721g9wma0s5
Image: 2018-03-13 20-34-08_SIM.jpeg, ID: ck1oa5z09x3990838zuzcj58d
Image: 2018-01-29 17-34-40_SIM.jpeg, ID: ck1wauhclv8og0811b1ei5ovq
Image: Incendio_10_R_17072018_103000_mp4_814, ID: ck2jnerf6wngt0748zmfgfr17
Image: Outras_sem_incendio_mp4_15, ID: ck2ialocotso407948m655h6v
Image: 2018-03-06 10-17-08_NAO.jpeg, ID: ck1xpzkk3iwgf07483cm9bz2k
Image: 2018-03-08 20-53-08_NAO.jpeg, ID: ck2me2goz4g7h0757g1d6bszx
Image: 2018-03-12 09-56-19_NAO.jpeg, ID: ck1oucc81igrs0811mp3jmif7
Image: 2018-03-20 15-47-50_NAO.jpeg, ID: ck28n9ou61t3x08385y7v9py1
Image: Outras_sem_incendio_mp4_173, ID: ck2inb7ksg9li0811iuuu7260
Image: 2019-02-08 08-2

Image: Outras_sem_incendio_mp4_676, ID: ck2i82iq8rtpk0748nyfd5mmm
Image: 2018-03-15 17-28-31_NAO.jpeg, ID: ck1yulx25hiyk0725ekv4ghjp
Image: Incendio_IP_10_R_17072018_103000_mp4_376, ID: ck2irfry6znqe0944c0mea7lu
Image: 2018-03-17 19-09-59_SIM.jpeg, ID: ck1qm15jewpdz07579s21bdmf
Image: 2018-03-02 11-54-00_NAO.jpeg, ID: ck1p3xukfjjvv0811fooucqhd
Image: 2018-03-04 10-07-05_NAO.jpeg, ID: ck1te25iwz0z50748zhmaw4fo
Image: 2018-03-02 09-38-19_NAO.jpeg, ID: ck224s4qj78im0757f3jjpge9
Image: 2018-03-13 15-54-21_NAO.jpeg, ID: ck1qvc7a8xxgt0721yut4bval
Image: 2018-03-07 20-48-21_SIM.jpeg, ID: ck1s6t66oid8708389r5a7hv8
Image: 2019-01-30 10-24-00_NAO.jpeg, ID: ck292n04k6m0e0944wsa7ewhp
Image: 2018-03-01 21-59-03_NAO.jpeg, ID: ck2mjgdy6uu5p074849dw3g9b
Image: 2018-03-17 09-27-35_NAO.jpeg, ID: ck1xfrf6iqhfh0721hxg8lr6o
Image: 2018-03-20 06-48-08_NAO.jpeg, ID: ck26dp6o5p8ph0944wn5vsm2g
Image: 2018-03-17 10-23-49_SIM.jpeg, ID: ck2j9f2f45t3y0757seh5ndpv
Image: 2018-03-09 22-22-54_SIM.jpeg, ID: ck1qnb68wn

Image: 2018-03-15 17-25-42_NAO.jpeg, ID: ck1tms43f5izd0838sk6ht742
Image: 2018-03-14 16-54-30_NAO.jpeg, ID: ck1tgaj96976u0721r54iiymu
Image: Incendio_IP_IP_30_R_19082018_142912_mp4_1173, ID: ck2i2w2r0r8g30748bnd4oyia
Image: 2018-03-14 17-31-17_NAO.jpeg, ID: ck2218cnl5vr9075732hu59h3
Image: Incendio_IP_10_R_17072018_103000_mp4_841, ID: ck2j237wl2mje0721pun7wwyi
Image: 2018-03-02 09-08-07_NAO.jpeg, ID: ck1owiz0jxudf0794oozb9m4g
Image: 2018-03-17 11-27-16_SIM.jpeg, ID: ck2j7t7uxynbf0838z8cr39mk
Image: 2018-10-29 11-20-43_SIM.jpeg, ID: ck29bpczttqxp0725r9fx8zqd
Image: 2018-03-16 09-45-18_NAO.jpeg, ID: ck1yyucq1yhnd0757tev42xr5
Image: 2018-03-21 14-47-50_SIM.jpeg, ID: ck23megi6qh8l0757ypxd2yjo
Image: 2019-07-04 13-29-48_NAO.jpeg, ID: ck27urzawog8j0725c5s5sfbl
Image: 2018-03-12 09-43-23_NAO.jpeg, ID: ck20voaonp9t70757q91uku2h
Image: 2018-03-16 11-55-32_NAO.jpeg, ID: ck1p2xh1qwarj0748wxsgqgiz
Image: 2019-09-13 07-11-52_NAO.jpeg, ID: ck2980d4n790i09442n2brpgo
Image: 2018-03-04 16-18-44_NAO.jpe

Image: Incendio_10_R_17072018_103000_mp4_706, ID: ck2ir6uqvuttl0794ong41kw4
Image: 2018-03-16 09-14-59_NAO.jpeg, ID: ck1omo0owhyeb0811oa4tjiyh
Image: 2018-03-20 13-54-56_NAO.jpeg, ID: ck2mktrmh1lo10944o2vg51tr
Image: 2018-03-16 12-41-31_NAO.jpeg, ID: ck1p4170fyrva07941w6lr0vf
Image: Outras_sem_incendio_mp4_849, ID: ck2i8zyeurxqh0748g9b3b7k5
Image: 2019-01-31 11-29-23_SIM.jpeg, ID: ck27w4puy5rh00757hwt7i6nq
Image: 2018-03-01 23-22-46_NAO.jpeg, ID: ck1vn7ffgx3h407949803ysc0
Image: 2018-03-08 06-58-40_SIM.jpeg, ID: ck1z1xnx8og900748y2pz6q56
Image: 2018-03-21 13-37-02_NAO.jpeg, ID: ck28cm0f47gmr0757jm1shrp7
Image: 2018-03-04 06-53-35_SIM.jpeg, ID: ck1qc0qz2mrxg072116x7xofk
Image: 2018-03-16 14-40-34_NAO.jpeg, ID: ck1ugqm6lil980838op6nzehi
Image: 2018-03-02 05-39-06_NAO.jpeg, ID: ck1owg9vzvit7074806rrs0s5
Image: 2018-03-12 17-41-18_NAO.jpeg, ID: ck20q06nxecu90794iv8i4ylc
Image: 2018-08-02 10-06-19_NAO.jpeg, ID: ck1nz5ddemtuq0725mq5my9xv
Image: 2019-06-13 22-55-37_NAO.jpeg, ID: ck2httcx1irvi